In [1]:
from sklearn import tree
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [2]:
beer_csv = pd.read_csv("beerlist_w_Kmean.csv")

In [3]:
beer_csv_short = beer_csv.loc[(beer_csv["review_overall"] > 3.5)& (beer_csv["review_aroma"] > 3.5) &\
                             (beer_csv["review_appearance"] > 3.5)& (beer_csv["review_palate"] > 3.5)&\
                             (beer_csv["review_taste"] > 3.5)]



In [4]:
beer_csv_short.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7297 entries, 5 to 13245
Columns: 282 entries, brewery_id to Kmeans Cluster
dtypes: float64(8), int64(271), object(3)
memory usage: 15.8+ MB


In [5]:
y = beer_csv["Kmeans Cluster"]

In [6]:
X = beer_csv[[
'grainy',
 'sweet',
 'hay',
 'straw',
 'cracker',
 'bicuity',
 'caramel',
 'toast',
 'roast',
 'coffee',
 'espresso',
 'burnt',
 'alcohol',
 'tobacco',
 'gunpowder',
 'leather',
 'pine',
 'grass',
 'raisins',
 'currant',
 'plum',
 'dates',
 'prunes',
 'figs',
 'blackberry',
 'blueberry',
 'banana',
 'pineapple',
 'apricot',
 'pear',
 'apple',
 'nectarine',
 'peach',
 'mango',
 'lemon',
 'lime',
 'orange',
 'tangerine',
 'clementine',
 'grapefruit',
 'zest',
 'metallic',
 'vinegar',
 'copper',
 'cidery',
 'astringent',
 'chlorine',
 'phenolic',
 'pepper',
 'clove',
 'anise',
 'licorice',
 'bacon',
 'fatty',
 'nutty',
 'butterscotch',
 'vanilla',
 'earthy',
 'woody',
 'bread',
 'saddle',
 'musty',
 'barnyard',
 'honey',
 'chestnut',
 'black',
 'garnet',
 'ruby',
 'brilliant',
 'hazy',
 'cloudy',
 'turbid',
 'opaque',
 'clear',
 'crystal',
 'bright',
 'dull',
 'persistent',
 'rocky',
 'large',
 'fluffy',
 'dissipating',
 'lingering',
 'white',
 'tan',
 'frothy',
 'delicate',
 'roasted',
 'bready',
 'bitter',
 'spicy',
 'fruity',
 'chocolate',
 'toffee',
 'tart',
 'subtle',
 'woodsy',
 'sulfuric',
 'assertive',
 'mild',
 'bold',
 'balanced',
 'robust',
 'intense',
 'harsh',
 'complex',
 'refined',
 'hearty',
 'smooth',
 'silky',
 'velvety',
 'prickly',
 'tingly',
 'creamy',
 'warming',
 'viscous',
 'oily',
 'Malty',
 'biscuity',
 'breadlike',
 'rich',
 'deep',
 'roasty',
 'cereal',
 'cookie',
 'coffeeish',
 'caramelly',
 'molasses',
 'complexity',
 'smoky',
 'autumnal',
 'cream',
 'milk',
 'oatmeal',
 'rustic',
 'layered',
 'piney',
 'citrusy',
 'grapefruity',
 'sharp',
 'fresh',
 'herbal',
 'zippy',
 'lemony',
 'aromatic',
 'floral',
 'springlike',
 'sprucelike',
 'juniper',
 'minty',
 'pungent',
 'elegant',
 'grassy',
 'baked',
 'bubblegum',
 'yeasty',
 'belgian',
 'tropical',
 'clean',
 'soft',
 'effervescent',
 'spritzy',
 'sparkling',
 'pinpoint',
 'bubbly',
 'gentle',
 'carbonation',
 'full',
 'light',
 'slick',
 'heavy',
 'dry',
 'thick',
 'thin',
 'country_Argentina',
 'country_Armenia',
 'country_Australia',
 'country_Austria',
 'country_Belgium',
 'country_Bosnia and Herzegovina',
 'country_Brazil',
 'country_Bulgaria',
 'country_Canada',
 'country_Cayman Islands',
 'country_China',
 'country_Colombia',
 'country_Costa Rica',
 'country_Czechia',
 'country_Denmark',
 'country_Ethiopia',
 'country_Finland',
 'country_France',
 'country_Funatsu',
 'country_Georgia',
 'country_Germany',
 'country_Greece',
 'country_Hong Kong',
 'country_India',
 'country_Indian Ocean',
 'country_Indonesia',
 'country_Ireland',
 'country_Israel',
 'country_Italy',
 'country_Jammu and Kashmir',
 'country_Japan',
 'country_Kenya',
 'country_Latvia',
 'country_Lebanon',
 'country_Liechtenstein',
 'country_Lithuania',
 'country_Luxembourg',
 'country_Madagascar',
 'country_Maihama',
 'country_Malaysia',
 'country_Mexico',
 'country_Monaco',
 'country_Myanmar',
 'country_Netherlands',
 'country_New Zealand',
 'country_Nigeria',
 'country_North Macedonia',
 'country_Norway',
 'country_Palestine',
 'country_Poland',
 'country_Portugal',
 'country_Puerto Rico',
 'country_Romania',
 'country_Russia',
 'country_Singapore',
 'country_Slovakia',
 'country_South Africa',
 'country_Spain',
 'country_Sri Lanka',
 'country_St Lucia',
 'country_Suriname',
 'country_Sweden',
 'country_Switzerland',
 'country_Tagus',
 'country_Tanzania',
 'country_The Bahamas',
 'country_Trinidad and Tobago',
 'country_Turks and Caicos Islands',
 'country_UK',
 'country_USA',
 'country_USVI',
 'country_Ukraine',
 'country_United States']]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=11)

In [8]:
model_1st = RandomForestClassifier(n_estimators = 100, n_jobs=17)

In [9]:
model_1st = model_1st.fit(X_train, y_train)

In [10]:
print(f"F1 Score Train: {model_1st.score(X_train, y_train):.4f}")
print(f"F1 Score Test: {model_1st.score(X_test, y_test):.4f}")

F1 Score Train: 1.0000
F1 Score Test: 0.6504


In [11]:
ft_names = X.columns

In [12]:
import_features = pd.Series(model_1st.feature_importances_, index = ft_names).sort_values(ascending=False)

In [13]:
import_features.head(50)

carbonation    0.044942
sweet          0.034408
light          0.030199
thin           0.026689
white          0.025462
smooth         0.024864
thick          0.021623
balanced       0.019267
dry            0.018303
clear          0.018158
earthy         0.016141
mild           0.015901
caramel        0.015774
creamy         0.015716
bitter         0.014521
orange         0.013666
bread          0.012980
alcohol        0.011734
heavy          0.011587
hazy           0.011382
fruity         0.011374
deep           0.010812
full           0.010763
tan            0.010173
rich           0.009774
floral         0.009414
complex        0.009310
clean          0.009176
bready         0.009091
subtle         0.008731
chocolate      0.008700
lingering      0.008683
herbal         0.008677
copper         0.008559
spicy          0.008310
apple          0.008073
fresh          0.007705
roasted        0.007674
toffee         0.007539
black          0.007472
cloudy         0.007118
pine           0

In [30]:
n_features = []

for i in range(25,60,5):
    n_features.append(i)

# define grid search
param_grid = {'max_features': n_features, 
              'n_estimators': [200, 250, 300],
              'max_depth': [20, 25, 30]
             }


In [32]:
model = RandomForestClassifier()

In [33]:
grid = GridSearchCV(model, param_grid, n_jobs = -1, verbose = 3)

In [34]:
grid.get_params(deep=True)

{'cv': None,
 'error_score': nan,
 'estimator__bootstrap': True,
 'estimator__ccp_alpha': 0.0,
 'estimator__class_weight': None,
 'estimator__criterion': 'gini',
 'estimator__max_depth': None,
 'estimator__max_features': 'auto',
 'estimator__max_leaf_nodes': None,
 'estimator__max_samples': None,
 'estimator__min_impurity_decrease': 0.0,
 'estimator__min_impurity_split': None,
 'estimator__min_samples_leaf': 1,
 'estimator__min_samples_split': 2,
 'estimator__min_weight_fraction_leaf': 0.0,
 'estimator__n_estimators': 100,
 'estimator__n_jobs': None,
 'estimator__oob_score': False,
 'estimator__random_state': None,
 'estimator__verbose': 0,
 'estimator__warm_start': False,
 'estimator': RandomForestClassifier(),
 'iid': 'deprecated',
 'n_jobs': -1,
 'param_grid': {'max_features': [25, 30, 35, 40, 45, 50, 55],
  'n_estimators': [200, 250, 300],
  'max_depth': [20, 25, 30]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': None,
 'verbose': 3}

In [35]:
grid.fit(X_train, y_train)

Fitting 5 folds for each of 63 candidates, totalling 315 fits


/opt/anaconda3/envs/PythonAdvanced/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 315 out of 315 | elapsed:  6.9min finished


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [20, 25, 30],
                         'max_features': [25, 30, 35, 40, 45, 50, 55],
                         'n_estimators': [200, 250, 300]},
             verbose=3)

In [36]:
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_estimator_)

{'max_depth': 30, 'max_features': 45, 'n_estimators': 300}
0.6632775680391833
RandomForestClassifier(max_depth=30, max_features=45, n_estimators=300)


In [37]:
model = RandomForestClassifier(max_depth=30, max_features=45, n_estimators=300)

In [38]:
model = model.fit(X_train, y_train)

In [39]:
import_features = pd.Series(model.feature_importances_, index = ft_names).sort_values(ascending=False)
import_features.head(50)

carbonation    0.081385
light          0.048877
sweet          0.044254
smooth         0.032106
white          0.031617
thin           0.028573
thick          0.023822
dry            0.019828
mild           0.016040
caramel        0.015033
bitter         0.014039
clear          0.012850
creamy         0.012814
bread          0.012058
balanced       0.011982
orange         0.011481
full           0.011049
tan            0.010666
deep           0.009652
earthy         0.009428
alcohol        0.009154
hazy           0.009149
floral         0.008812
bready         0.008681
fruity         0.008599
rich           0.008587
heavy          0.008343
clean          0.008115
copper         0.007784
pine           0.007525
spicy          0.007455
black          0.007359
herbal         0.007296
complex        0.007247
subtle         0.007151
roasted        0.007106
chocolate      0.007043
fresh          0.007003
toffee         0.006969
lingering      0.006944
cloudy         0.006776
tart           0